In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
from tqdm import tqdm
from math import log10
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [0]:
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold, GridSearchCV, KFold

In [0]:
df = pd.read_csv('dba.csv')

In [93]:
df.head()

,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [94]:
df = shuffle(df).reset_index(drop=True)
train_data = df[472:]
test_data = df[:472]
print(train_data.shape, test_data.shape)

(900, 5) (472, 5)


In [0]:
def get_c_range(x_train, y_train):
    c_ = np.logspace(-5, 8, 10)
    scores = []
    for c in c_:
        svc = LinearSVC(penalty='l1', C=c, dual=False)
        svc.fit(x_train, y_train)
        scores.append(svc.score(x_train, y_train))
    scores = np.array(scores)
    ind = np.argwhere(scores > 0.9).flatten()
    c_1 = c_[ind[0]]
    c_2 = c_[ind[-1]]
    return c_1, c_2

In [0]:
def linearSVC(x_train, y_train):
    c_l, c_h = get_c_range(x_train, y_train)
    parameters = {'C':np.logspace(log10(c_l), log10(c_h), 20)}

    svc = LinearSVC(penalty='l1', dual=False)
    n_splits = 10 if (len(x_train) > 10) else 5

    kf = StratifiedKFold(n_splits=n_splits, shuffle=True)
    clf = GridSearchCV(svc, parameters, cv=kf, scoring='accuracy', return_train_score=True)
    clf.fit(x_train, y_train)
    return clf.best_estimator_

In [102]:
results1 = []
test_data = shuffle(test_data)
x_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]
train_data = shuffle(train_data)
print(train_data)
train_nottak, train_taken = train_test_split(train_data, test_size=10)
print(train_nottak, train_taken)
while(len(train_taken['class'].unique()) == 1 or train_taken['class'].value_counts().min() < 2):
  train_nottak, train_taken = train_test_split(train_data, test_size=10)
each = []
for j in range(20):
  sv = linearSVC(train_taken.iloc[:, :-1], train_taken.iloc[:, -1])
  print(sv)
  if(j < 19):
    dist = sv.decision_function(train_nottak.iloc[:, :-1])
    print("Returns the decision function of the sample for each class in the model.")
    print(dist)
    
    train_nottak['dist'] = dist
    print("Returns the decision function of X for each class in the model.")
    print(train_nottak['dist'])
    
    train_nottak = train_nottak.sort_values(['dist'], ascending=True)
    print("Add Value")
    print(train_nottak)
    
    train_taken = train_taken.append(train_nottak.iloc[:10, :-1])
    print("Verification of the 10-element sample")
    print(train_taken)
    
    train_nottak = train_nottak.iloc[10:, :-1]
    print(train_nottak)
  each.append(1 - sv.score(x_test, y_test))
results1.append(np.mean(each))

      variance   skewness   curtosis   entropy  class
835   3.230300   7.838400  -3.534800 -1.215100      0
603  -2.123400   1.181500  -0.555520 -0.811650      1
633   4.656200   7.639800  -2.424300 -1.238400      0
1339  0.927030   9.431800  -0.662630 -1.672800      0
1240 -0.814790  -5.738100   4.391900  0.321100      1
1332  1.011700   0.902200  -2.350600  0.427140      1
1070 -3.142300 -13.036500  15.677300 -0.661650      1
971   5.681000   7.795000  -2.684800 -0.925440      0
797  -2.221400  -0.237980   0.560080  0.056020      1
1309  4.650000  -4.829700   3.455300 -0.251740      0
520  -0.697450  -1.767200  -0.344740 -0.123720      1
1261  3.802300  -3.869600   4.044000  0.953430      0
1195  2.059700  -0.993260   5.211900 -0.293120      0
508   2.422600  -4.575200   5.947000  0.215070      0
987  -2.215300  11.962500   0.078538 -7.785300      0
894   3.624400   1.460900   1.350100  1.928400      0
1347 -2.208300  -9.106900   8.999100 -0.284060      1
1119 -2.967200 -13.286900  1

In [0]:
results1 = []
test_data = shuffle(test_data)
x_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]
    
for i in tqdm(range(50)):
    train_data = shuffle(train_data)
    train_nottak, train_taken = train_test_split(train_data, test_size=10)
    while(len(train_taken['class'].unique()) == 1 or train_taken['class'].value_counts().min() < 2):
        train_nottak, train_taken = train_test_split(train_data, test_size=10)
    
    each = []
    
    for j in range(90):
        sv = linearSVC(train_taken.iloc[:, :-1], train_taken.iloc[:, -1])
        
        if(j < 89):
            dist = sv.decision_function(train_nottak.iloc[:, :-1])
            train_nottak['dist'] = dist
            train_nottak = train_nottak.sort_values(['dist'], ascending=True)
            train_taken = train_taken.append(train_nottak.iloc[:10, :-1])
            train_nottak = train_nottak.iloc[10:, :-1]
        
        each.append(1 - sv.score(x_test, y_test))
    results1.append(np.mean(each))
pd.DataFrame(results1).to_csv('Error.csv', index=False)